In [1]:
import string
import numpy as np
import pandas as pd
import codecs
import joblib
import pymorphy3
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from annoy import AnnoyIndex

import warnings
warnings.filterwarnings('ignore')


In [13]:
df = pd.read_csv("./data/ProductsDataset.csv")
df.rename(columns={'descrirption': 'description', 'product_id' : 'answer'}, inplace=True)
data_market = df[['title', 'description', 'answer']]

data_market['text'] = data_market['title']
data_market.drop(['title', 'description'], axis=1, inplace=True)

data_market.dropna(inplace=True)

data_market.head()

,answer,text
0,58e3cfe6132ca50e053f5f82,Юбка детская ORBY
1,5667531b2b7f8d127d838c34,Ботильоны
2,59534826aaab284cba337e06,Брюки
3,57de544096ad842e26de8027,Продам детские шапки
4,5ad4d2626c86cb168d212022,Блузка


In [14]:
morpher = pymorphy3.MorphAnalyzer()

stop_words = set(stopwords.words('russian'))
sw = set(stop_words)


def preprocess_txt(line):
    if isinstance(line, str):
        exclude = set(string.punctuation)
        line = line.replace(',', ', ')
        line = line.replace('.', '. ')
        line = line.replace('не ', 'не')
        spls = "".join(i for i in line if i not in exclude).strip().split()
        spls = [morpher.parse(i.lower())[
            0].normal_form for i in spls if i.lower() not in sw and i != ""]
        
        spls = ' '.join(spls)
        spls = spls.split(' ')
        
        return spls
    else:
        return str(line)


data_market['text'] = data_market['text'].apply(preprocess_txt)
data_market['class'] = 1
data_market.head()

,answer,text,class
0,58e3cfe6132ca50e053f5f82,"[юбка, детский, orby]",1
1,5667531b2b7f8d127d838c34,[ботильон],1
2,59534826aaab284cba337e06,[брюки],1
3,57de544096ad842e26de8027,"[продать, детский, шапка]",1
4,5ad4d2626c86cb168d212022,[блузка],1


In [15]:
data_talker = pd.read_csv(
    './data/prepared_answers.txt', sep='\t',
    header=None, names=['text', 'answer'], 
    on_bad_lines="skip"
)

data_talker.dropna(inplace=True)
data_talker['class'] = 0

data_talker = data_talker.iloc[:len(data_market)]

data_talker['text'] = data_talker['text'].apply(preprocess_txt)

data_talker.head()

,text,answer,class
1,"[парень, относиться, цветной, линза, девушка, ...",меня вобще прикалывает эта тема :).,0
2,"[делать, сегодня, найти, 2, миллион, рубль]","Если это ""счастье "" действительно на вас свали...",0
3,"[эбу, двенашка, называться, итэлма, эбу]",ЭБУ — электронный блок управления двигателем а...,0
4,"[академия, вампир, сколько, даный, момент, час...","4. Охотники и Жертвы, Ледяной укус, Поцелуй ть...",0
5,"[защититься, энергетический, вампир]",Защита мыслью. <br>Каждый человек должен в отн...,0
